In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np

df = pd.read_csv('2024_Wimbledon_featured_matches.csv')

In [2]:
# 简洁查看
print("列名:", df.columns.tolist()[:15])  # 先看前15列
print("\n缺失值百分比 (>0%):")
missing = (df.isnull().sum() / len(df) * 100)
print(missing[missing > 0].round(2))

列名: ['match_id', 'player1', 'player2', 'elapsed_time', 'set_no', 'game_no', 'point_no', 'p1_sets', 'p2_sets', 'p1_games', 'p2_games', 'p1_score', 'p2_score', 'server', 'serve_no']

缺失值百分比 (>0%):
speed_mph       10.32
serve_width      0.74
serve_depth      0.74
return_depth    17.97
dtype: float64


In [3]:
# 检查定类数据列
categorical_cols = ['winner_shot_type', 'serve_width', 'serve_depth', 'return_depth']
print("检查定类数据列:")
for col in categorical_cols:
    if col in df.columns:
        print(f"{col}: 存在 ✓ - 唯一值数: {df[col].nunique()}")
    else:
        print(f"{col}: 不存在 ✗")

检查定类数据列:
winner_shot_type: 存在 ✓ - 唯一值数: 3
serve_width: 存在 ✓ - 唯一值数: 5
serve_depth: 存在 ✓ - 唯一值数: 2
return_depth: 存在 ✓ - 唯一值数: 2


In [4]:
# ========== 1. 定类数据处理 ==========
print("="*50)
print("1. 处理定类数据")
print("="*50)

categorical_cols = ['winner_shot_type', 'serve_width', 'serve_depth', 'return_depth']
df_processed = df.copy()

# 用众数填充缺失值
for col in categorical_cols:
    missing_count = df_processed[col].isnull().sum()
    if missing_count > 0:
        mode_value = df_processed[col].mode()[0]
        print(f"{col}: 缺失值 {missing_count} 个，用众数 '{mode_value}' 填充")
        df_processed[col].fillna(mode_value, inplace=True)

# 一-hot编码
df_categorical_encoded = pd.get_dummies(
    df_processed[categorical_cols],
    columns=categorical_cols,
    drop_first=False,
    prefix=categorical_cols
)

print(f"\none-hot编码后列数: {df_categorical_encoded.shape[1]}")
print("编码后列名示例:", df_categorical_encoded.columns.tolist()[:5])

1. 处理定类数据
serve_width: 缺失值 54 个，用众数 'C' 填充
serve_depth: 缺失值 54 个，用众数 'NCTL' 填充
return_depth: 缺失值 1309 个，用众数 'ND' 填充

one-hot编码后列数: 12
编码后列名示例: ['winner_shot_type_0', 'winner_shot_type_B', 'winner_shot_type_F', 'serve_width_B', 'serve_width_BC']


C:\Users\Mily\AppData\Local\Temp\ipykernel_22588\2363150449.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_processed[col].fillna(mode_value, inplace=True)


In [ ]:
# ========== 2. 定量数据处理 ==========
print("\n" + "="*50)
print("2. 处理定量数据")
print("="*50)

# 获取所有数值列（排除已处理的定类列和ID列）
id_cols = ['match_id', 'player1', 'player2', 'set_no', 'game_no', 'point_no']
exclude_cols = categorical_cols + id_cols + ['p1_sets', 'p2_sets', 'p1_games', 'p2_games', 'p1_score', 'p2_score']

numeric_cols = [col for col in df_processed.columns 
                if df_processed[col].dtype in [np.float64, np.int64] and col not in exclude_cols]

print(f"\n发现 {len(numeric_cols)} 列定量数据:")
print(numeric_cols)

# 准备数据：以match_id分组
df_numeric_filled = df_processed[id_cols + numeric_cols].copy()

print("\n全局均值:")
global_means = {}
for col in numeric_cols:
    global_means[col] = df_numeric_filled[col].mean()
    print(f"{col}: {global_means[col]:4f}")

print("\n按match_id分组处理缺失值:")
print("="*50)

# 按match_id分组处理
for match_id in df_numeric_filled['match_id'].unique():
    print(f"\n比赛 {match_id}:")
    df_match_idx = df_numeric_filled['match_id'] == match_id
    
    for col in numeric_cols:
        # 计算该group内该列的缺失比例
        missing_count = df_numeric_filled[df_match_idx][col].isnull().sum()
        group_size = df_match_idx.sum()
        missing_pct_group = (missing_count / group_size * 100) if group_size > 0 else 0
        
        if missing_pct_group > 20:
            print(f"  {col}: 缺失 {missing_count}/{group_size} ({missing_pct_group:.2f}%) > 20%, 跳过 ✗")
        elif missing_pct_group > 0:
            # 用全局均值填充
            global_mean = global_means[col]
            df_numeric_filled.loc[df_match_idx, col] = df_numeric_filled.loc[df_match_idx, col].fillna(global_mean)
            print(f"  {col}: 缺失 {missing_count}/{group_size} ({missing_pct_group:.2f}%), 用全局均值 {global_mean:.4f} 填充 ✓")

print("\n\n数据处理完成!")


2. 处理定量数据

发现 29 列定量数据:
['server', 'serve_no', 'point_victor', 'p1_points_won', 'p2_points_won', 'game_victor', 'set_victor', 'p1_ace', 'p2_ace', 'p1_winner', 'p2_winner', 'p1_double_fault', 'p2_double_fault', 'p1_unf_err', 'p2_unf_err', 'p1_net_pt', 'p2_net_pt', 'p1_net_pt_won', 'p2_net_pt_won', 'p1_break_pt', 'p2_break_pt', 'p1_break_pt_won', 'p2_break_pt_won', 'p1_break_pt_missed', 'p2_break_pt_missed', 'p1_distance_run', 'p2_distance_run', 'rally_count', 'speed_mph']

全局均值:
server: 1.5103
serve_no: 1.3607
point_victor: 1.4896
p1_points_won: 63.4942
p2_points_won: 62.3480
game_victor: 0.2416
set_victor: 0.0233
p1_ace: 0.0475
p2_ace: 0.0439
p1_winner: 0.1716
p2_winner: 0.1624
p1_double_fault: 0.0167
p2_double_fault: 0.0181
p1_unf_err: 0.1263
p2_unf_err: 0.1359
p1_net_pt: 0.1043
p2_net_pt: 0.1285
p1_net_pt_won: 0.0748
p2_net_pt_won: 0.0875
p1_break_pt: 0.0380
p2_break_pt: 0.0312
p1_break_pt_won: 0.0139
p2_break_pt_won: 0.0104
p1_break_pt_missed: 0.0242
p2_break_pt_missed: 0.0207
p1_d

In [6]:
# ========== 3. 合并数据并验证 ==========
print("\n" + "="*50)
print("3. 合并数据并验证")
print("="*50)

# 合并数据
df_final = pd.concat([df_processed[id_cols], df_categorical_encoded, df_numeric_filled[numeric_cols]], axis=1)

print(f"\n最终数据集形状: {df_final.shape}")
print(f"  - ID列: {len(id_cols)}")
print(f"  - 定类数据(one-hot编码): {df_categorical_encoded.shape[1]} 列")
print(f"  - 定量数据: {len(numeric_cols)} 列")
print(f"  - 总计: {df_final.shape[1]} 列")

# 验证缺失值
total_missing = df_final.isnull().sum().sum()
print(f"\n缺失值验证: {total_missing} 个缺失值")
if total_missing == 0:
    print("✓ 所有处理过的数据都已填补完毕!")
else:
    print("⚠ 仍存在缺失值（来自缺失比例>20%的列）")

# 保存处理后的数据
df_final.to_csv('2024_Wimbledon_featured_matches_washed.csv', index=False)
print("\n✓ 数据已保存到 '2024_Wimbledon_featured_matches_washed.csv'")


3. 合并数据并验证

最终数据集形状: (7284, 47)
  - ID列: 6
  - 定类数据(one-hot编码): 12 列
  - 定量数据: 29 列
  - 总计: 47 列

缺失值验证: 488 个缺失值
⚠ 仍存在缺失值（来自缺失比例>20%的列）

✓ 数据已保存到 '2024_Wimbledon_featured_matches_washed.csv'
